## Convolving light-curves 

The script convolves the magnification maps to getting the microlensed signal.

1. Setting the path for the directories and input parameters
2. Function to calculate the difference between A, B images
3. Functions to convolve the data
4. Execution of convolving the maps and saving the data

Rewritten by: Sophie Stucki <br>
Script originally written by: Soumya Shreeram & Eric Paic <br>
Date: 20nd Juni 2021 <br>

In [26]:
import numpy as np
from astropy.io import fits
from astropy.convolution import convolve_fft
import scipy.signal as ss

from time import sleep
import os,sys


In [27]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. Setting the path for the directories and input parameters

In [28]:
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
data_dir = os.path.join(root_dir, "/content/drive/MyDrive/Colab Notebooks/TP4b")
print("Does the directory exist? \n>",os.path.isdir(data_dir))
print(data_dir)

# setting the paths
datadir = os.path.join(data_dir,  'Data')
resultdir = os.path.join(datadir,  'results')
mapdir = os.path.join(datadir,  'maps', 'unconvolved')
storagedir = os.path.join(datadir,  'maps', 'storage')

Does the directory exist? 
> True
/content/drive/MyDrive/Colab Notebooks/TP4b


All input parameters:

In [29]:
# Type of binning: none, each pixel or nbr_r per category with a uniform distribution
bin = 2
if bin == 0:
    list_r0 = [2,4,8,12,20,30,40,60,80,100]
    #intervals of r
    interval_r = [[1,2],[3,5],[6,10],[11,15],[16,25],[26,35],[36,55],[56,65],[66,85],[86,125]]
elif bin == 1:
    list_r0 = np.linspace(1,125,125)
    #intervals of r
    interval_r = [[1,2],[3,5],[6,10],[11,15],[16,25],[26,35],[36,55],[56,65],[66,85],[86,125]]
elif bin == 2:
    list_r0 = [2,4,8,12,20,30,40,60,80,100]
    #intervals of r
    interval_r = [[1,2],[3,5],[6,10],[11,15],[16,25],[26,35],[36,55],[56,65],[66,85],[86,125]]
    nbr_r = 10
else:
    print("Error: the value of bin is not valid.")

list_img = ['A','B']

list_comb = [('A', 'B')]

einstein_r = 3.414e16
cm_per_pxl = (20*einstein_r)/8192
ld_per_pxl = 30000000000*3600*24/cm_per_pxl

dim = 512

### 2. Function to calculated the difference between the A, B images

In [30]:
def mapDiff(mapA,mapB,r0,comb, idx=-1):
    """
    Function takes the logarithmic difference between the two maps A and B
    Input:
    @mapA, mapB :: two randomly chosen maps
    @r0 :: scale radius
    @comb :: combination of the two maps eg. A1, B2 or A3, B2, etc.
    """
    img = fits.open(mapA)[0]
    map_A = img.data[:, :]

    img = fits.open(mapB)[0]
    map_B = img.data[:, :]

    final_map = map_A/map_B

    hdu = fits.PrimaryHDU(final_map)
    os.chdir(data_dir)
    if idx==-1:
      hdu.writeto(resultdir+'/map%s-%s_fml09_R%s_thin_disk_fml09.fits'%(comb[0],comb[1],r0))
    else:
      hdu.writeto(resultdir+'/map%s-%s_fml09_R%s_%s_thin_disk.fits'%(comb[0],comb[1],r0,idx))
    return final_map

### 3. Functions to convolve the data

The models of the light source used to convolve the mgnification maps are:
* `thin_disk` (default option) more information available [here](https://arxiv.org/pdf/1707.01908.pdf).
* `sersic`
* `thin_disk&node`
* `wavy_hole`
* `wavy`
* `sersic`

In [31]:
def calR(x, xc, y, yc):
    "Function calculates the radius: (xc, yc) are the centre of the disk"
    return np.sqrt((x-xc)**2 + (y-yc)**2)

def calXi(r, Rin, R0):
    "Function checks the value of r and calculates the light profile \Xi accordingly"
    if r<Rin:
        return 0
    else:
        if Rin == 0: # for J0158 since event horizon not resolvable
            xi = (r / R0) ** (3/4) * (1) ** (-1/4)
        else: # Shakura Sunyavev light distribution for a thin disk
            xi = (r/R0)**(3/4)*(1-np.sqrt(Rin/r))**(-1/4)
    return xi

def getprofilevalue(x, y, xc, yc, xn, yn, I0, R0, model, Rin):
    """
    Function to get the profile values for each particular choice of model 
    @(x,y) :: position on the map
    @(xc, yc) :: position of the centre of the quasar disk
    @(xn, yn) :: position of the arbitary node
    @I0, R0 :: intensity, I0, at scale radius, R0
    @model :: choice among 'thin_disk', 'thin_disk&node', 'wavy', etc.
    @Rin :: inner radius of the accreting disk
    """
    # parameters for 'wavy' type models
    n, reff_pix, sersic_index =3, 0.2, 4.0
    beta = n*np.pi/(2*R0)
    
    if model == "thin_disk":
      r = calR(x, xc, y, yc)
      xi = calXi(r, Rin, R0)
      profile_val = I0/(np.exp(xi)-1)
        
    if model == "thin_disk&node":
      r = calR(x, xn, y, yn)
      xi = calXi(r, Rin, R0)
      profile_val = I0/(np.exp(xi)-1) + I0/(np.exp((r/(R0*0.4106))**(3/4))-1)

    if model =="wavy_hole":
      r = calR(x, xc, y, yc)
      # calculates the profile value based on conditions on beta*r
      if beta*r < 2*np.pi:
          profile_val = I0*beta/(n*np.pi**2)*np.power(np.sin(beta*r),2)/r
      else:
          profile_val = 0

    if model =="wavy":
      r = calR(x, xc, y, yc)
      if beta*r < np.pi/2:
          profile_val = I0*beta/(n*np.pi**2)
      elif beta * r < 2 * np.pi and beta * r > np.pi/2:
          profile_val = I0*beta/(n*np.pi**2)*np.square(np.sin(beta*r))
          
    if model == "sersic":
      r = calR(x, xc, y, yc)
      profile_val = I0 * np.exp(-(r / reff_pix) ** (1.0 / sersic_index))
    return profile_val

def generate2DDataArrays(dims=128):
    """
    Function to generate the 2D data arrays as per required dims
    @dims :: sets the size of the 2D data arrays
    """    
    data = np.ones((dims,dims))
    pdata = np.ones((dims,dims))
    gdata = np.ones((dims,dims))
    return data, pdata, gdata

def get2dgaussianvalue(x, y, xc, yc, sigma):
    "Convolving the pixels with a Gaussian function (only if necessary)"
    return 1.0 / (2 * np.pi * sigma ** 2) * np.exp(-((x - xc) ** 2 + (y - yc) ** 2) / (2 * sigma ** 2))

def get2ddiracvalue(x, y, xc, yc):
    "Convolving the pixels with a Dirac function"
    if x == xc and y ==yc:
        return 1
    else:
        return 0
    
def outputMapParams(img):
    "Function calculates some of the magnification map paramters"
    map_d = img.data[:, :]
    macro_mag = np.mean(map_d)
    map_d = map_d / macro_mag
    return map_d, macro_mag

def blurProfile(pdata, gdata, data):
    """
    Convolves the data and blurs the light profile
    """
    output = convolve_fft(gdata, pdata, normalize_kernel=True)

    for lind, line in enumerate(data):
        for cind, elt in enumerate(line):
            data[lind][cind] = output[lind][cind]
    return data

def convolve(R0, map_name, model="thin_disk", Rin=1e-9, I0=1, blur=False, interval=False,i=-1, cat_R0=0):
    """
    Input:
    @map_name :: fits file of the magnification map
    @model :: model of the light source that the magnification map will be convoluted with
    @R0 :: scale radius, for thin_disk, R0 ~10^14 cm for 1131,0435 (Units: pixels)
    @Rin :: inner radius of the disk (useful only for thin_disk)
    @I0 :: Intensity at scale radius
    
    Returns:: writes data to folder 'storagedir'
    """
    img_name = map_name.split('map')[2].split('.')[0]
    img = fits.open(map_name)[0]
    map_d, macro_mag  = outputMapParams(img)
    
    
    # centre of thin disk, poistion of other node
    xc, yc, xn, yn = 256, 256, 128, 128

    # generates the profile, gaussian, and toconv fits files from new_canvas.fits
    data, pdata, gdata = generate2DDataArrays(dims=512)

    # fills the data arrays based of the choice of the model
    for lind, line in enumerate(gdata):
        for cind, elt in enumerate(line):
            gdata[lind][cind] = get2ddiracvalue(cind+1, lind+1, xn, yn)
            pdata[lind][cind] = getprofilevalue(cind+1, lind+1, xc, yc, xn, yn, I0, R0, model, Rin)
            
    if blur:
        pdata = blurProfile(pdata, gdata, data)

    #normalization
    pdata = pdata/ np.sum(pdata) 
      
    # final convolution
    out2 = ss.fftconvolve(map_d, pdata, mode="valid") 
    hdu = fits.PrimaryHDU(out2)
    
    if interval == False:
      hdu.writeto(storagedir+'/convolved_map_%s_fft_%s_%i.fits'%(img_name,model,R0))
    else:
      hdu.writeto(storagedir+'/convolved_map_%s_fft_%s_%i_%i.fits'%(img_name,model,cat_R0,i))
    return

def getFilename(rootdir, string_name, params, param_arr=True, interval=False):
    """
    Function generates the filenames for reading/writing out data
    @rootdir, string_name :: root directory containing the file, file name
    @params :: parameters that distinguish the file name
    """
    if interval==False:
      if param_arr:
          filename1 = os.path.join(rootdir, string_name%(params[0], params[2]))
          filename2 = os.path.join(rootdir, string_name%(params[1], params[2]))
          return filename1, filename2
      else:
        return os.path.join(rootdir, string_name%(params))
    else:
      if param_arr:
          filename1 = os.path.join(rootdir, string_name%(params[0], params[2],params[3]))
          filename2 = os.path.join(rootdir, string_name%(params[1], params[2],params[3]))
          return filename1, filename2
      else:
        return os.path.join(rootdir, string_name%(params))
    
def showProgress(idx, n):
    """
    Function prints the progress bar for a running function
    @param idx :: iterating index
    @param n :: total number of iterating variables/ total length
    """
    j = (idx+1)/n
    sys.stdout.write('\r')
    sys.stdout.write("[%-20s] %d%%" % ('='*int(20*j), 100*j))
    sys.stdout.flush()
    sleep(0.25)
    return 

### 4.1 Convolving the maps and saving the data

In [32]:
if bin == 0 or bin == 1:
    for img_AB in list_img:
        for idx, r0 in enumerate(list_r0):
            # generate filename
            filename = getFilename(mapdir, 'map%s.fits', img_AB, param_arr=False)
        
            # convolve the maps
            convolve(r0, filename,"thin_disk", blur=False)

        # shows progress
        showProgress(idx, len(list_r0))

In [34]:
if bin == 2:
    for img_AB in list_img:
       for idx, r0 in enumerate(list_r0):
            R=np.linspace(interval_r[idx][0],interval_r[idx][1],nbr_r)
            for idx_r, r in enumerate(R):       
                # generate filename
                filename = getFilename(mapdir, 'map%s.fits', img_AB, param_arr=False)
                # convolve the maps
                convolve(r, filename,"thin_disk", blur=False, interval=True, i=idx_r, cat_R0=r0)

            # shows progress
            showProgress(idx, len(list_r0)) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in true_divide


OSError: ignored

### 4.2 Calculates the differences in the magnification maps that are convolved

In [ ]:
if bin == 0 or bin == 1:
    string_name = "convolved_map_%s_fft_thin_disk_%s_fml09.fits"
    for i,comb in enumerate(list_comb):
        for r0 in list_r0:
            r0= int(r0)
            #generate filenames for saving it on disk
            params = [comb[0], comb[1], r0]
            filename1, filename2 = getFilename(storagedir, string_name, params, param_arr=True)

            # calculates the difference between the maps
            final_map = mapDiff(filename1, filename2, r0, comb)

        # shows progress
        showProgress(i, len(list_comb))

In [ ]:
if bin == 2:
    string_name = "convolved_map_%s_fft_thin_disk_%s_%s.fits"
    for i,comb in enumerate(list_comb):
       for r0 in list_r0:
            for idx in range(nbr_r-1):
                #generate filenames for saving it on disk
                params = [comb[0], comb[1], r0, idx]
                filename1, filename2 = getFilename(storagedir, string_name, params, param_arr=True, interval=True)

                # calculates the difference between the maps
                final_map = mapDiff(filename1, filename2, r0, comb, idx=idx)
            # shows progress
            showProgress(i, len(list_comb))